<a href="https://colab.research.google.com/github/LordRelentless/NGFTSimulations/blob/main/Simulation_5_4_Engineered_Spacetime_Potential_via_an_S_%CE%BC%CE%BD_Field_Objective_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Simulation 5.4: Engineered Spacetime Potential via an S_μν Field
# Language: Julia

using Plots
using Printf
using LinearAlgebra
gr()

println("--- NGFT Engineered Spacetime Simulation ---")

# --- 1. Simulation & NGFT Parameters ---
global N_FRAMES = 500
global DT = 0.5
global LATTICE_SIZE = 200

# Central Mass (I_μν Source)
global GRAVITATIONAL_STRENGTH = 1.0 # Strength of the I_μν potential well
global CENTER_MASS_POS = [100.0, 100.0]

# Engineered BEC (S_μν Source)
global NUM_VORTICES = 6
global VORTEX_RADIUS = 30.0 # Radius of the BEC boundary
global ENTROPIC_PRESSURE_STRENGTH = 1.2 # Strength of the S_μν potential hill
                                      # Must be > GRAVITATIONAL_STRENGTH to cause repulsion

# Test Particle
global PARTICLE_START_POS = [50.0, 125.0]

# --- 2. Helper Functions ---
function generate_gaussian(size, center, sigma)
    x_coords = [i for i in 1:size, j in 1:size]
    y_coords = [j for i in 1:size, j in 1:size]
    return exp.(-((x_coords .- center[1]).^2 .+ (y_coords .- center[2]).^2) ./ (2 * sigma^2))
end

function get_gradient(field, pos)
    ix, iy = round(Int, pos[1]), round(Int, pos[2])
    if ix < 2 || ix > size(field, 1)-1 || iy < 2 || iy > size(field, 2)-1
        return [0.0, 0.0]
    end
    grad_x = (field[ix+1, iy] - field[ix-1, iy]) / 2.0
    grad_y = (field[ix, iy+1] - field[ix, iy-1]) / 2.0
    return [grad_x, grad_y]
end

# --- 3. Setup Potentials and Particle ---
# I_μν Field: The attractive potential well from the central mass
global potential_attractive = -GRAVITATIONAL_STRENGTH .* generate_gaussian(LATTICE_SIZE, CENTER_MASS_POS, 25.0)

# S_μν Field: The repulsive potential hill from the engineered BEC vortices
global potential_repulsive = zeros(LATTICE_SIZE, LATTICE_SIZE)
global vortex_positions = zeros(NUM_VORTICES, 2)
for i in 1:NUM_VORTICES
    angle = 2pi * i / NUM_VORTICES
    vortex_pos = CENTER_MASS_POS + [VORTEX_RADIUS * cos(angle), VORTEX_RADIUS * sin(angle)]
    vortex_positions[i, :] = vortex_pos
    global potential_repulsive .+= ENTROPIC_PRESSURE_STRENGTH .* generate_gaussian(LATTICE_SIZE, vortex_pos, 8.0)
end

# Net Potential Field is the sum of the two, as per the NGFT field equations
global net_potential = potential_attractive .+ potential_repulsive

# Initialize particle
global particle_pos = copy(PARTICLE_START_POS)
global particle_vel = [1.0, -0.2] # Give it an initial velocity towards the well

# --- 4. PHASE 1: Run Simulation ---
println("Phase 1: Calculating particle trajectory...")
global path_history = [copy(particle_pos)]

for _ in 1:N_FRAMES
    # The particle's acceleration is determined by the gradient of the NET potential
    accel = -get_gradient(net_potential, particle_pos)

    global particle_vel += accel * DT
    global particle_pos += particle_vel * DT

    push!(path_history, copy(particle_pos))
end
println("Phase 1 complete.")

# --- 5. PHASE 2: Render Animation ---
println("Phase 2: Rendering animation...")
global anim = @animate for i in 1:2:N_FRAMES
    current_plot = plot(xlims=(0, LATTICE_SIZE), ylims=(0, LATTICE_SIZE), aspect_ratio=:equal,
         size=(1000, 1000), background_color=:black,
         title="Engineered Spacetime Potential (Frame $i/$N_FRAMES)",
         legend=false, grid=false, xticks=false, yticks=false)

    # Plot the net potential field
    heatmap!(current_plot, 1:LATTICE_SIZE, 1:LATTICE_SIZE, net_potential', c=:coolwarm, colorbar=false)

    # Plot the BEC boundary
    bec_boundary_angles = range(0, 2pi, length=100)
    bec_x = CENTER_MASS_POS[1] .+ (VORTEX_RADIUS + 10) .* cos.(bec_boundary_angles)
    bec_y = CENTER_MASS_POS[2] .+ (VORTEX_RADIUS + 10) .* sin.(bec_boundary_angles)
    plot!(current_plot, bec_x, bec_y, ls=:dash, color=:cyan, alpha=0.7)

    # Plot the S_μν sources (vortices)
    scatter!(current_plot, vortex_positions[:,1], vortex_positions[:,2], marker=:utriangle, color=:cyan, markersize=8)

    # Plot the I_μν source (central mass)
    scatter!(current_plot, [CENTER_MASS_POS[1]], [CENTER_MASS_POS[2]], marker=:circle, color=:yellow, markersize=10)

    # Plot the particle path and current position
    path_to_draw = path_history[1:i]
    plot!(current_plot, [p[1] for p in path_to_draw], [p[2] for p in path_to_draw], color=:lightgreen, lw=2)
    scatter!(current_plot, [particle_pos[1]], [particle_pos[2]], color=:lightgreen, markersize=6)

    current_plot # Return the plot for the animation
end

# --- 6. Save Final Outputs ---
gif(anim, "ngft_engineered_potential.gif", fps=30)
println("Animation saved as ngft_engineered_potential.gif")

# Save the last frame as a static image
global final_plot = plot(xlims=(0, LATTICE_SIZE), ylims=(0, LATTICE_SIZE), aspect_ratio=:equal,
         size=(1000, 1000), background_color=:black,
         title="Engineered Spacetime Potential (Final Frame)",
         legend=false, grid=false, xticks=false, yticks=false)

heatmap!(final_plot, 1:LATTICE_SIZE, 1:LATTICE_SIZE, net_potential', c=:coolwarm, colorbar=false)

bec_boundary_angles = range(0, 2pi, length=100)
bec_x = CENTER_MASS_POS[1] .+ (VORTEX_RADIUS + 10) .* cos.(bec_boundary_angles)
bec_y = CENTER_MASS_POS[2] .+ (VORTEX_RADIUS + 10) .* sin.(bec_boundary_angles)
plot!(final_plot, bec_x, bec_y, ls=:dash, color=:cyan, alpha=0.7)

scatter!(final_plot, vortex_positions[:,1], vortex_positions[:,2], marker=:utriangle, color=:cyan, markersize=8)

scatter!(final_plot, [CENTER_MASS_POS[1]], [CENTER_MASS_POS[2]], marker=:circle, color=:yellow, markersize=10)

path_to_draw = path_history[1:end]
plot!(final_plot, [p[1] for p in path_to_draw], [p[2] for p in path_to_draw], color=:lightgreen, lw=2)
scatter!(final_plot, [particle_pos[1]], [particle_pos[2]], color=:lightgreen, markersize=6)


savefig(final_plot, "ngft_engineered_potential_final.png")
println("Final plot saved as ngft_engineered_potential_final.png")
println("Simulation successfully finished.")

--- NGFT Engineered Spacetime Simulation ---
Phase 1: Calculating particle trajectory...
Phase 1 complete.
Phase 2: Rendering animation...


Animation saved as ngft_engineered_potential.gif


[ Info: Saved animation to /content/ngft_engineered_potential.gif


Final plot saved as ngft_engineered_potential_final.png
Simulation successfully finished.
